In [14]:
!pip install pinecone-client==2.2.4
!pip install sentence-transformers
!ctransformers==0.2.5
!sentence-transformers==2.2.2
!pinecone-client==2.2.4
!langchain==0.0.225
!flask
!python-dotenv

'ctransformers' is not recognized as an internal or external command,
operable program or batch file.
'sentence-transformers' is not recognized as an internal or external command,
operable program or batch file.
'pinecone-client' is not recognized as an internal or external command,
operable program or batch file.
Traceback (most recent call last):
  File "C:\Users\USER\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\USER\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\USER\AppData\Local\Programs\Python\Python310\Scripts\langchain.exe\__main__.py", line 4, in <module>
  File "C:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\langchainplus_sdk\__init__.py", line 4, in <module>
    from langchainplus_sdk.client import LangChainPlusClient
  File "C:\Users\USER\AppData\Local\Programs\Python\Py

Usage: flask [OPTIONS] COMMAND [ARGS]...

  A general utility script for Flask applications.

  An application to load must be given with the '--app' option, 'FLASK_APP'
  environment variable, or with a 'wsgi.py' or 'app.py' file in the current
  directory.

Options:
  -e, --env-file FILE   Load environment variables from this file, taking
                        precedence over those set by '.env' and '.flaskenv'.
                        Variables set directly in the environment take highest
                        precedence. python-dotenv must be installed.
  -A, --app IMPORT      The Flask application or factory function to load, in
                        the form 'module:name'. Module can be a dotted import
                        or file path. Name is not required if it is 'app',
                        'application', 'create_app', or 'make_app', and can be
                        'name(args)' to pass arguments.
  --debug / --no-debug  Set debug mode.
  --version             Sh

Error: Could not locate a Flask application. Use the 'flask --app' option, 'FLASK_APP' environment variable, or a 'wsgi.py' or 'app.py' file in the current directory.

'python-dotenv' is not recognized as an internal or external command,
operable program or batch file.


In [34]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [16]:

PINECONE_API_KEK= "pcsk_6fWHkW_KEpJNQ8FNZGkQdT5ne2du9DqnrPGJjwJJvAA1JU9qEtpn3TjqScvGihy2hr4bcc"
PINECONE_API_ENV = "gcp-starter"

In [17]:
## Extract data from the PDF

def load_pdf(data):
    loader = DirectoryLoader(data,glob ='*.pdf',loader_cls = PyPDFLoader)

    documents = loader.load()

    return documents

In [18]:
extracted_data = load_pdf("C:/Users/USER/Projects/Medical Chatbot/data/")

In [6]:
#extracted_data

In [19]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size =500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [20]:
text_chunks = text_split(extracted_data)
print("length of my chunk", len(text_chunks))

length of my chunk 5860


In [9]:
#text_chunks

In [21]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [29]:

!pip install sentence-transformers


In [30]:
embeddings = download_hugging_face_embeddings()

ImportError: Could not import sentence_transformers python package. Please install it with `pip install sentence-transformers`.

In [ ]:
import sentence_transformers
print(sentence_transformers.__version__)


In [ ]:
query_result = embeddings.embed_query("hello world")
print("Length",len(query_result))

In [ ]:
# Initializing the Pinecone
pinecone.init(api_key=PINECONE_API_KEK,environment=PINECONE_API_ENV)

index_name = "medical-chatbot"

docsearch = Pinecone.from_texts([t.page_content for t in text_chunks],embeddings,  index_name = index_name)

In [ ]:
docsearch =Pinecone.from_exixtiog_index(index_name,embeddings)

query = "What are Allergies"

docs = docsearch.similarity_search(query,k=3)

print("Result",docs)

In [ ]:
prompt_template = """
  Use the following pieces of information  to answer the user's question.
  If you don't know the answer , just say that you don 't know, don'  try to make up an answer.

  Context:{context} 
  Question:{question}

  Only return the helpful answer below and nothing else.
  Helpful answer:
"""

In [ ]:
PROMPT = PromptTemplate(template=prompt_template,input_variables=["context",'question'])
chain_type_kwarge={"prompt":PROMPT}

In [ ]:
llm= CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",model_type="llama",colfig={'max_new_tokens':512,'temperature':0.8})

In [ ]:
qa=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever = docsearch.as_retriever(search_kwargs={'k':2})
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwarge
)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result = qa({"query":user_input})
    print("Response:",result["result"])